# Source reconstruction with EEG


`
Author: Britta Westner
`

In this notebook, we will have a look at source reconstruction with EEG data. We will try different source reconstruction techniques on the `sample` data, and compare the results. Along the way, we will learn about some important things to consider when source-reconstructing EEG data.

In [1]:
%matplotlib inline
import mne

mne.set_log_level('warning')

# set paths:
sample_path = mne.datasets.sample.data_path()
data_path = sample_path / 'MEG' / 'sample'
subjects_dir = sample_path / 'subjects'

## Read raw and re-reference to the average

First, we read the raw data from disk, and keep only the EEG data (and the `stim` channel, to be able to cut our data in epochs). 

In [ ]:
fname_raw = data_path / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(fname_raw)
raw.pick(picks=['eeg', 'stim']).load_data()

For the source reconstruction of EEG data, it's vital to use an **average reference**. If you were to also compute a forward model (which we skip here for time reasons and simply load from disk), it is important to re-reference your data to the average _before_ computing the forward model. The referencing scheme of the forward model always has to match that of the data!

Read an explanation of why we use an average reference below.

<div class="alert alert-block alert-info">
    <b>Average reference in source reconstruction</b>
    <br> <br>
      "The forward field is to be calculated with the same reference as the sensor space data. An average reference is a good choice, as it mitigates the modelling error introduced by localization inaccuracies of the electrodes’ positions. With a single reference electrode the model error of this electrode will be passed to all other electrodes, and this error decreases through averaging across all electrodes." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>

In [ ]:
# set the average reference for the EEG data
raw.set_eeg_reference(ref_channels='average', projection=True)

## Compute epochs and evoked potentials

Next, we want to create epochs from the continuous data. Here, we use the trials with an auditory stimulus delivered to the left or right ear. That was coded with trigger value `1` and `2`, respectively. We average those epochs to create an evoked potential, which we then plot to look at.

In [ ]:
events = mne.find_events(raw)
epochs = mne.Epochs(raw, events, event_id=[1, 2])
evoked = epochs.average()
evoked.plot_joint();

## The forward model for EEG

Due to the time constraints of the workshop, we load the forward model from disk. If you process your own data, it is important to know that you **cannot** use a single shell forward model for EEG (as is fine to use with MEG).  

For EEG, we need to model the brain, skull, and scalp, as all three play a pivotal role in the spread of the currents from the source to the electrodes. Just modelling the brain is not enough. Most commonly used nowadays is a 3-layer Boundary Element Model (BEM), but  3-layer Finite Element Models (FEM) are used as well. Those are more computationally intensive, though, and need a lot longer to compute.

If you want to learn more about how to compute forward models, check out the notebook `Forward_modelling.ipynb`.

<div class="alert alert-block alert-info">
    <b>EEG forward models</b>
    <br> <br>
      "[T]he accuracy of the forward field is of high importance, as EEG is very sensitive to the head conduction profile (Hamalainen and Sarvas, 1987; Hillebrand and Barnes, 2005; Steinsträter et al., 2010; Wolters et al., 2006). This necessitates the use of a realistic head model, e.g., a boundary element or finite element model. This model should ideally be created from the individual MRI scan of the participant to maximize accuracy of the forward model." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


In [7]:
# load the precomputed EEG forward model from disk
fname_fwd = data_path / 'sample_audvis-eeg-oct-6-fwd.fif'
fwd = mne.read_forward_solution(fname_fwd)

## Compute a minimum norm estimate of the data

Let's first compute a source reconstruction using minimum norm estimation (MNE). This technique estimates the source strength across the cortical surface. For MNE, we need a few ingredients:

- _covariance matrix_: This covariance matrix should represent "noise", thus we are taking the pre-stimulus baseline. Another prominent choice is an empty room measurement (if available).
- _the data_: The inverse operator will be applied to this - we pass in our evoked object we have created above.
- _the forward model_: The one we loaded from disk.
- _method choice_: We also have to choose the "flavour" of MNE. dSPM (dynamical Statistical Parametric Mapping), which scales the data based on a noise estimate.

In [8]:
from mne.minimum_norm import make_inverse_operator, apply_inverse
# A quick note on imports: usually, you would have all imports at the beginning of your script.
# Here, we choose to break this rule to help you keep track of where functions come from.

noise_cov = mne.compute_covariance(epochs, tmax=0.0)
inv = make_inverse_operator(evoked.info, fwd, noise_cov)
stc_mne = apply_inverse(evoked, inv, method='dSPM')

Now we can visualize the result. We choose to crop the data a bit on the time axis since we don't expect the auditory evoked potential to stretch beyond the first 200 ms. 

You can press Play to see the spread of activation.

The time courses in the lower window show the maxmimum activity in the left (blue trace) and right hemisphere (orange trace). You can see the locations of the maxima in the brain plot, marked with a blue and orange dot. 

You can click on the brain to add the time courses of other locations to the time window. 

In [ ]:
stc_mne.crop(-0.01, 0.2).plot(hemi='both', subjects_dir=subjects_dir, subject='sample')

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Is the activity where you expected it to be? </li>
      <li> You can compare different MNE estimates by changing the method parameter in the apply_inverse() step. See below for more information! </li>
    </ul>
</div>

### Other MNE methods

The "flavour" of MNE is chosen in `apply_inverse()`. You can easily compare the dSPM solution to other solutions by changing that parameter and re-running the step.

Other methods are plain Minimum Norm Estimation (`MNE`), standardized LOw Resolution brain Electromagnetic TomogrAphy (`sLORETA`) and the exact LORETA (`eLORETA`). You will see that not all methods perform equally well on this data set.

## Prepare beamforming of data

Another source reconstruction is beamforming. Beamforming is more sensitive to forward model inaccuracies (especially for EEG!) than MNE solutions, but it can provide more focal localizations than MNE solutions.
For Beamforming to work, we also need a high density of EEG electrodes. A rule of thumb that is used sometimes is to have more than 60 electrodes. In this data set, we have _exactly_ 60 - so let's give it a try but be cautious and inspect the outcome carefully!

For the computation of the beamformer, we need a **data covariance matrix**. This is different from MNE: the data covariance matrix is computed including the _data of interest_, not across noise. 
Thus, beamformers are adaptive to the data!

Let's compute our covariance matrix over the first 200 ms after stimulus onset. 

If you want to know more about covariance matrices and their estimation, check out the second section of the `Beamforming_best_practices.ipynb` notebook or the `Working_with_SSSed_data.ipynb` notebook.

In [10]:
data_cov = mne.compute_covariance(epochs, tmin=0., tmax=0.2,
                                  method='empirical', rank='info')

It's good practice to visualize our data covariance matrix. You also get an estimate of the rank in the singular value plot.

In [ ]:
mne.viz.plot_cov(data_cov, info=epochs.info);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> You can see that we are rank-deficient by 2 (60 channels, but only a rank of 58). Do you know why? Hint: look at the output of epochs.info </li>
    </ul>
</div>

We explicitly save the rank for the EEG data in a dictionary, so we can pass it to the beamformer:

In [12]:
ranks = {'eeg': 58}

## Compute beamformer and apply to evoked data

Now we can compute the beamformer (sometimes also called a spatial filter) and apply it to the evoked. We choose the Linearly Constrained Minimum Variance beamformer (LCMV), which is expecting time-resolved data, such as an evoked potential.

We use the following ingredients to compute the beamformer:
- _covariance matrix_: This covariance matrix should represent the data.
- _the data_: The beamformer will be applied to this - we pass in our evoked object we have created above.
- _the forward model_: The one we loaded from disk.
- _orientation_: We ask the beamformer to compute the source orientations that maximize power for us. That is done using `pick_ori='max-power'`.
- _rank_: We pass the rank of the data, so that the beamformer can take care of any rank-deficiencies internally.

In [13]:
from mne.beamformer import make_lcmv, apply_lcmv

filters = make_lcmv(epochs.info, fwd, data_cov=data_cov, pick_ori='max-power', rank=ranks)
stc_lcmv = apply_lcmv(evoked=evoked, filters=filters)

We can plot the brain and time course using the same way we did before. Do the results match our dSPM results?

In [14]:
stc_lcmv.crop(-0.01, 0.2).plot(subjects_dir=subjects_dir, subject='sample', hemi='both');

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Do the results match the ones from the dSPM reconstruction? </li>
      <li> Do they match your expecations of source activity? </li>
    </ul>
</div>

## Take home messages

Here are the top things to pay special attention to when source reconstructing EEG data:

- Use an average reference and make sure the forward model is also using an average reference!
- Use a 3-layer realistic head model, e.g. a BEM or FEM.
- Source reconstruction (especially with beamformers) works best with a higher-density EEG setup. 
- It's good to check your outcomes carefully (always, not only with EEG). This can include using a _sanity check_, i.e., a easily-identifiable source such as an auditory or visual evoked potential before moving on to more complex contrasts in the data (this is something to already anticipate when designing your experiment!) or to compare your output to that of other source reconstruction techniques.